<a href="https://colab.research.google.com/github/elichen/karpathyGPT/blob/main/Let's_build_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
block_size = 8
batch_size = 32
learning_rate = 2e-3
max_inters = 3000
eval_iters = 200
eval_interval = 300
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
!wget -q https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [4]:
text = open("input.txt").read()

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab_size, "".join(chars)

(65, "\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")

In [6]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

In [7]:
data = torch.tensor(encode(text))
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x.to(device), y.to(device)

In [9]:
class BigramModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        return embeds

    def calculate_loss(self, inputs, targets):
        logits = self(inputs)
        return F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1))

    def generate(self, input_batch, max_new_tokens):
        generated = input_batch
        for _ in range(max_new_tokens):
            logits = self(generated)
            last_logits = logits[:, -1, :]
            probabilities = F.softmax(last_logits, dim=-1)
            next_tokens = torch.multinomial(probabilities, 1).squeeze(1)
            generated = torch.cat((generated, next_tokens.unsqueeze(1)), dim=1)
        return generated

In [10]:
@torch.no_grad()
def estimate_loss(model):
  out = {}
  model.eval()
  for split in ["train", "val"]:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      loss = model.calculate_loss(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [11]:
model = BigramModel(vocab_size).to(device)
xb,yb = get_batch("train")
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

for steps in range(max_inters):
  if steps % eval_interval == 0:
    losses = estimate_loss(model)
    print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb,yb = get_batch("train")
  loss = model.calculate_loss(xb,yb)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
print(loss.item())

step 0: train loss 4.5277, val loss 4.5246
step 300: train loss 3.9060, val loss 3.9156
step 600: train loss 3.4309, val loss 3.4460
step 900: train loss 3.1126, val loss 3.1297
step 1200: train loss 2.9040, val loss 2.9187
step 1500: train loss 2.7650, val loss 2.7857
step 1800: train loss 2.6843, val loss 2.6860
step 2100: train loss 2.6176, val loss 2.6341
step 2400: train loss 2.5780, val loss 2.5951
step 2700: train loss 2.5564, val loss 2.5694
2.641012668609619


In [12]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=400)[0].tolist()))


Rirosengeanerved, ncve!
Paceacks.
Hachot BR:
KandwthinfZjad t, t fo, licks ved

R:V: t QmerthisitXPrud nghJp:
T:avilo f fl arard;

TNOfCor t ut Onoboullanet hthXtok y, wXEN h,
Thate qKNCiomillXmusecoQ'd-war I he
O,
A:

Thy thespalealeto menont.RO,
ithind s mThe: hPlR:JHeplPLefongucais ar wlagnou.

s ndmy ft lW:NowalldgrcouWTat tonERDFOREd mesery-d h arsobepreXEnck himGrte wit feePa ty d es I IN:qk


In [15]:
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [42]:
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1]
    xbow[b,t] = torch.mean(xprev, 0)
xbow.shape

torch.Size([4, 8, 2])

In [43]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x
torch.allclose(xbow, xbow2)

True

In [50]:
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [44]:
xbow[0], xbow2[0]

(tensor([[ 0.0048, -1.8912],
         [-0.5169, -1.0678],
         [ 0.1308, -0.8717],
         [ 0.0418, -0.7062],
         [ 0.2783, -0.4006],
         [ 0.4349, -0.2755],
         [ 0.1695, -0.3922],
         [ 0.1996, -0.2580]]),
 tensor([[ 0.0048, -1.8912],
         [-0.5169, -1.0678],
         [ 0.1308, -0.8717],
         [ 0.0418, -0.7062],
         [ 0.2783, -0.4006],
         [ 0.4349, -0.2755],
         [ 0.1695, -0.3922],
         [ 0.1996, -0.2580]]))

In [38]:
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
a, b, c

(tensor([[1.0000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000],
         [0.3333, 0.3333, 0.3333]]),
 tensor([[0., 7.],
         [2., 7.],
         [5., 5.]]),
 tensor([[0.0000, 7.0000],
         [1.0000, 7.0000],
         [2.3333, 6.3333]]))

tensor([[1.],
        [2.],
        [3.]])